# Recency bias  

Prepare data for the case studies
* Compute normalized primacy and recency (filter data if needed)
* Visualize


In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# reload local modules automatically
%load_ext autoreload 
%autoreload 2  
import utils as up   

### Global parameters:

In [ ]:
# filter out data that doesn't meet these conditions:
filter_length = 30 # only keep if there are filter_length reptitions or more (>=)
filter_score = 3 # only scores bigger than filter_score (>)
filter_lag = 9  # only lags smaller than filter_lag (<)

# Compute normalized primacy and recency
Run this code for each case study

In [ ]:
case_study = "case_study_1"
my_date_session_letters = ['1', '2', '3', '4','5', '6', '7', '8','9', '10']

#case_study = "case_study_2"
#my_date_session_letters = ['11', '12', '13', '14','15', '16']

#case_study = "case_study_3"
#my_date_session_letters = ['17', '18', '19', '20']

### Process and normalize data

In [ ]:
url = "https://api.github.com/repos/nlihin/R2R-analysis/contents/" + case_study + "/rankings"
csv_files = up.get_files(url)

In [ ]:
csv_file_list = sorted(
        csv_files,
        key=lambda url: int(url.rsplit("/", 1)[-1].split("_", 1)[0])
    )

combine to one file

In [ ]:
final_combined_df = up.process_files(csv_file_list)

normalize

In [ ]:
final_combined_df = up.normalize_PR(final_combined_df, case_study)

pivot

In [ ]:
grouped_df = final_combined_df.groupby(["score", "lag"])[["lag"]].count().rename(columns={"lag": "count"})
grouped_df = grouped_df.reset_index()
pivot_df = grouped_df.pivot(index="lag", columns="score", values="count")
pivot_df

replace session numbers 1..n with real session letters/numbers

In [ ]:
final_combined_df = up.replace_session_ids(final_combined_df, my_date_session_letters)

In [ ]:
len(final_combined_df)

In [ ]:
save_full = final_combined_df.copy()
save_full["user_id"] = save_full.username.astype(str) + "_" + save_full.session_id.astype(str)
save_full.insert(0, "user_id", save_full.pop("user_id"))
save_full.drop(columns=["username"], inplace=True)

In [ ]:
len(save_full)

In [ ]:
file_path = "output/" + case_study + "/" + case_study + "PR_all_data.csv"

if os.path.exists(file_path):
    os.remove(file_path)

save_full.to_csv(file_path, index=False, encoding='utf-8-sig')

In [ ]:
save_full

In [ ]:
#statistics
print("total data: ", len(save_full[["user_id"]]))
print("sessions: ", len(save_full["session_id"].unique()))

### Filter out data that doesn't meet conditions

In [ ]:
filtered_df = save_full.groupby(["score","lag"]).filter(lambda x: len(x) >= filter_length)
filtered_df.to_csv("output/" + case_study + "/" + case_study + "_PR_norm_filter_data.csv", index = False, encoding = 'utf-8-sig', mode='w')

In [ ]:
len(filtered_df)

In [ ]:
norm_df = filtered_df[(filtered_df.score> filter_score) & (filtered_df.lag< filter_lag)]

In [ ]:
print("total data: ", len(save_full[["user_id"]]))
print("sessions: ", len(save_full["session_id"].unique()))

# Vizualitzations

In [ ]:
up.viz_lineplots(norm_df)

In [ ]:
all_df1 = pd.read_csv("output/" + case_study + "/" + case_study +"PR_all_data.csv")

scores = [5, 4, 3]
cpal = "BrBG"
up.viz_barplot(case_study, all_df1[all_df1.lag<10], scores, "output/" + case_study, cpal)  #works with filtered_df

In [ ]:
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold
import numpy as np

# Define the formula for the mixed model
#formula = 'pr_score ~ extra + neuro + score + lag'
#formula = 'pr_score ~ extra + neuro + score + lag + extra*neuro + score*lag'
#formula = 'pr_score ~ extra + neuro + score + lag + I(extra**2) + I(lag**2)'



# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a list to store R^2 scores
r2_scores = []

# Perform cross-validation manually for mixed linear model
for train_index, test_index in kf.split(df):
    train_data = df.iloc[train_index]
    test_data = df.iloc[test_index].copy()  # Make a deep copy of the test set
    
    # Fit the mixed linear model on training data
    model = smf.mixedlm(formula, data=train_data, groups=train_data['username'])
    result = model.fit()
    
    # Predict on the test data
    test_data['predicted'] = result.predict(test_data)
    
    # Calculate R^2 for the test set
    ss_total = sum((test_data['pr_score'] - test_data['pr_score'].mean())**2)
    ss_res = sum((test_data['pr_score'] - test_data['predicted'])**2)
    r2 = 1 - (ss_res / ss_total)
    r2_scores.append(r2)

# Output the results
print(f"Cross-validation R^2 scores for mixed model: {r2_scores}")
print(f"Average R^2 score for mixed model: {np.mean(r2_scores)}")


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create dataframe for VIF calculation
X = df[['extra', 'neuro', 'score', 'lag']]
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


In [ ]:
# Drop 'score' to reduce multicollinearity
formula = 'pr_score ~ extra + neuro + lag'
model = smf.mixedlm(formula, data=df, groups=df['username'])
result = model.fit()
print(result.summary())


In [ ]:
pip install pygam


In [ ]:
import pandas as pd
from pygam import LinearGAM, s

# Prepare the data
X = df[['extra', 'neuro', 'lag']]
y = df['pr_score']

# Fit the GAM model
gam = LinearGAM(s(0) + s(1) + s(2)).fit(X, y)

# Summary of the model
print(gam.summary())


In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Fit the final model on the full dataset
final_gam = LinearGAM(s(0) + s(1) + s(2)).fit(X, y)

# Predict and calculate residuals
y_full_pred = final_gam.predict(X)
residuals = y - y_full_pred

# Residuals vs Fitted plot
plt.scatter(y_full_pred, residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.show()

# QQ plot for normality
sm.qqplot(residuals, line='s')
plt.title('QQ Plot of Residuals')
plt.show()


In [ ]:
# Assuming your response variable is `pr_score`
df['sqrt_pr_score'] = np.sqrt(df['pr_score'])

# Fit the model with the transformed response variable
model_sqrt = smf.mixedlm('sqrt_pr_score ~ extra + neuro + lag', data=df, groups=df['username'])
result_sqrt = model_sqrt.fit()
print(result_sqrt.summary())



#### SVM

In [ ]:
filter_df = df[df.lag<20]

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

X = filter_df[['extra', 'agree', 'consciene', 'neuro', 'open', 'lag', 'score']]
y = filter_df['pr_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svr = SVR(kernel='rbf').fit(X_train, y_train)

print(f'R^2 score: {svr.score(X_test, y_test)}')


#### Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = filter_df[['extra', 'agree', 'consciene', 'neuro', 'open', 'lag', 'score']]
y = filter_df['pr_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

print(f'Feature importances: {rf.feature_importances_}')
print(f'R^2 score: {rf.score(X_test, y_test)}')


Q for Erez: what atr the cutoffs?

In [ ]:
#cutoffs = {'E': 3, 'A': 3, 'C': 3, 'N': 3, 'O': 3}

In [ ]:
traits = ['extra', 'agree','consciene', 'neuro', 'open']

In [ ]:
cutoffs = {}

for trait in traits:
    cutoffs[trait] = df[trait].median()

cutoffs

In [ ]:
def do_discrete(df, cutoffs1):
    for trait, cutoff in cutoffs1.items():
        df[f"{trait.lower()}_high"] = df[trait] > cutoff
    
    formula = 'pr_score ~ extra_high + agree_high + consciene_high + neuro_high + open_high'

    model = ols(formula, data=df).fit()

    anova_table = sm.stats.anova_lm(model, typ=2)

    print(anova_table)
    print(model.summary())

In [ ]:
do_discrete(df, cutoffs)

In [ ]:
corr_matrix_high = df[['score', 'lag', 'total_comparisons', 'pr_score', 'extra_high', 'agree_high',
       'consciene_high', 'neuro_high', 'open_high']].corr(method='spearman')

In [ ]:
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix_high.round(2), annot=True, cmap='coolwarm', mask=mask, cbar=True)
plt.show()

open is significant and with highest PR.  
now the same but for agg data:

In [ ]:
cutoffs = {}

for trait in traits:
    cutoffs[trait] = df_grouped[trait].median()

In [ ]:
do_discrete(df_grouped, cutoffs)

not any good

#### vizualize

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i,trait in enumerate(traits):
    sns.lineplot(data=df, x='lag', y='pr_score', hue=f'{trait.lower()}_high', ax = axes[i])
    plt.title(f'{trait}')

plt.tight_layout()
plt.show()


In [ ]:
formula = 'pr_score ~ extra + agree + consciene + neuro + open + lag + score'
model = ols(formula, data=df).fit()
print(model.summary())

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

X = df_grouped[['extra', 'agree', 'consciene', 'neuro', 'open', 'lag']]
y = df_grouped['pr_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lasso = Lasso(alpha=0.1).fit(X_train, y_train)
print(f'Lasso coefficients: {lasso.coef_}')
print(f'R^2 score: {lasso.score(X_test, y_test)}')


In [ ]:
from sklearn.linear_model import LassoCV

# Automatically tunes the regularization strength using cross-validation
lasso_cv = LassoCV(cv=5).fit(X, y)
print(lasso_cv.alpha_)  # Optimal regularization strength
print(lasso_cv.coef_)   # Coefficients with optimal alpha


In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)  # Adjust alpha as needed
ridge.fit(X, y)
print(ridge.coef_)


In [ ]:
from sklearn.model_selection import cross_val_score
r2_score = ridge.score(X, y)
print(f"R² score: {r2_score}")

In [ ]:
from sklearn.linear_model import RidgeCV
ridge_cv = RidgeCV(alphas=[10, 20, 50, 100, 120,150])
ridge_cv.fit(X, y)
print(f"Best alpha: {ridge_cv.alpha_}")


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[['extra', 'agree', 'consciene', 'neuro', 'open', 'lag', 'score']]
X_scaled = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
pca_components = pca.fit_transform(X_scaled)

print(f'Explained variance by PCA components: {pca.explained_variance_ratio_}')
